In [89]:
import cv2
import os
import time
import numpy as np
import itertools
from scipy.spatial import distance

os.chdir("/home/david/Downloads")

ORI_X = 0
ORI_Y = 1
WIDTH = 2
HEIGHT = 3
people_height = 1.7

PERSON_CLASS_ID = 0
COLOR_PEOPLE_BOX = (99,  43, 206)
COLOR_FAR_LINE = (102, 220, 225)
COLOR_CLOSE_LINE = (36,60, 255)
LIGHT_COLOR_TEXT = (255,255,255)
DARK_COLOR_TEXT = (0, 0, 0)
THICKNESS_LINE = 2
SECURE_DISTANCE = 2


def distance_measure(boxes):
    
    print(box)
    
    get_lower_center = lambda box: (box[ORI_X] + box[WIDTH] // 2, box[ORI_Y] + box[HEIGHT])
    f = lambda x: np.arctan(x) / (np.pi/2)

    results = []
    for (index_a, box_a), (index_b, box_b) in itertools.combinations(enumerate(boxes), 2):
        base_box_a = get_lower_center(box_a)
        #print("base_box_a",box_a,base_box_a)
        base_box_b = get_lower_center(box_b)
        #print("base_box_b",box_b,base_box_b)
        euclidean_distance = distance.euclidean(base_box_a, base_box_b)
        #print("euclidean_distance",euclidean_distance)
        height_box_a = people_height / box_a[HEIGHT]
        #print("height box a",height_box_a)
        height_box_b = people_height / box_b[HEIGHT]
        #print("height box b",height_box_b)
        l1 = f(box_a[HEIGHT] / box_b[HEIGHT])
        l2 = 1 - l1
        D = l1 * height_box_a * euclidean_distance + l2 * height_box_b * euclidean_distance
        a = [index_a, index_b,base_box_a, base_box_b,round(D, 2)]
        results.append(a)
    return np.array(results)


def get_min_distances(distances):
    min_distances = []
    if len(distances)>0:
        for i in set(distances[:,0]):
            a = distances[distances[:,0] == i][distances[distances[:,0] == i][:,-1].argmin()]
            min_distances.append(a)

    return np.array(min_distances)


def calculate_statistical_results(boxes, distance_lines):
    amount_people   = len(boxes)

    if amount_people > 1:
        minimal_distance = min([line[4] for line in distance_lines])
        breaking_secure_distance = sum([line[4] < SECURE_DISTANCE for line in distance_lines])
        average_distance = np.mean([line[4] for line in distance_lines])
    else:
        minimal_distance = 0
        breaking_secure_distance = 0
        average_distance = 0

    return {
            'amount_people': amount_people,
            'breaking_secure_distance': breaking_secure_distance,
            'minimal_distance': minimal_distance,
            'average_distance': average_distance,
    }


def draw_over_frame(image, boxes, distance_lines):
    for box in boxes:
        edge_0 = (box[ORI_X], box[ORI_Y])
        edge_1 = (box[ORI_X] + box[WIDTH], box[ORI_Y] + box[HEIGHT])
        cv2.rectangle(image, edge_0, edge_1, COLOR_PEOPLE_BOX, THICKNESS_LINE)

    for line in distance_lines:
        
        print("line",line)
        
        line_color = COLOR_CLOSE_LINE if line[4] < SECURE_DISTANCE else COLOR_FAR_LINE
        image = cv2.line(image, line[2], line[3], line_color, THICKNESS_LINE)

        e = ((np.array(line[2])+np.array(line[3]))/2).astype(int)
        mesure_text = '{}m'.format(line[4])
        position_text = (e[0], e[1] - 5)
        cv2.putText(image, mesure_text, position_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, DARK_COLOR_TEXT, THICKNESS_LINE)
        cv2.putText(image, mesure_text, position_text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, LIGHT_COLOR_TEXT, THICKNESS_LINE - 1)

    return image

In [90]:
#weightsPath = "/opt/dist2/distancia2-api/yolo-coco/yolov3.weights"
#configPath  = "/home/david/Downloads/yolov3.cfg"
#labelspath  = "/opt/dist2/distancia2-api/yolo-coco/coco.names"
#net = cv2.dnn.readNetFromDarknet(configPath,weightsPath)


net = cv2.dnn.readNetFromDarknet("yolov3.cfg","yolov3.weights")
#net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
#net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA) 
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA_FP16)

#ln  = net.getLayerNames()
ln = net.getLayerNames()
print(ln)

#ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
### sin soporte para CUDA
ln = [ln[i - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

#print(net.getUnconnectedOutLayers())
classes = open('coco.names').read().strip().split('\n')
np.random.seed(42)
colors = np.random.randint(0, 255, size=(len(classes), 3), dtype='uint8')

cap = cv2.VideoCapture(0)
success, frame = cap.read()

while success:

    #cv2.imshow('Camara orig', frame)
    
    ### transformando la imagen en otro tamanio
    #frame = cv2.resize(frame, (640,360),fx=0,fy=0,interpolation=cv2.INTER_LINEAR) 
    #frame = cv2.resize(frame, (416,416),fx=0,fy=0,interpolation=cv2.INTER_LINEAR) 
    
    #cv2.imshow('Camara resized', frame)
    
    ## INICIO DE HACE CAJAS
    h,w = frame.shape[:2]
    #print(H,W)
    
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416,416), swapRB=True, crop=False)
    
    net.setInput(blob)
    
    start = time.time()
    outputs = net.forward(ln)
    end = time.time()
    
    print(end-start)
    #r = blob[0, 0, :, :]
    #cv2.imshow('blob', r)
    #text = f'Blob shape={blob.shape}'
    #cv2.displayOverlay('blob', text)
    
    boxes = []
    confidences = []
    classIDs = []

    for output in outputs:
        print(output.shape)
        for detection in output:
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]
            if confidence > 0.5:
                box = detection[:4]*np.array([w,h,w,h])
                (centerX, centerY, width, height) = box.astype("int")
                x = int(centerX - (width / 2))
                y = int(centerY - (height / 2))
                box = [x, y, int(width), int(height)]
                boxes.append(box)
                confidences.append(float(confidence))
                classIDs.append(classID)
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.1)
    
    if len(indices) > 0:
        for i in indices.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in colors[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 4)
            text = "{}: {:.4f}".format(classes[classIDs[i]], confidences[i])
            cv2.putText(frame, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    idxs = np.array(sorted(cv2.dnn.NMSBoxes(boxes, confidences, 0.8, 0.1)))
    
    #print(idxs)
    if len(idxs) > 0:
        boxes = np.array(boxes)[idxs[:]]
    else:    
        boxes = np.array([])
    
    dist_lines = distance_measure(boxes)
    shorter_distance =  get_min_distances(dist_lines)
    stat = calculate_statistical_results(boxes,shorter_distance)
    
    print("stat['amount_people']",stat['amount_people'])
    if stat['amount_people'] > 0:
        frame = draw_over_frame(frame, boxes, shorter_distance)

    cv2.imshow('window', frame)
        
    #print(dist_lines)
    #print(classIDs)
    #break
    
    if cv2.waitKey(1) == ord('q'):
        break    
        
    success, frame = cap.read()

cap.release()
cv2.destroyAllWindows()


('conv_0', 'bn_0', 'leaky_1', 'conv_1', 'bn_1', 'leaky_2', 'conv_2', 'bn_2', 'leaky_3', 'conv_3', 'bn_3', 'leaky_4', 'shortcut_4', 'conv_5', 'bn_5', 'leaky_6', 'conv_6', 'bn_6', 'leaky_7', 'conv_7', 'bn_7', 'leaky_8', 'shortcut_8', 'conv_9', 'bn_9', 'leaky_10', 'conv_10', 'bn_10', 'leaky_11', 'shortcut_11', 'conv_12', 'bn_12', 'leaky_13', 'conv_13', 'bn_13', 'leaky_14', 'conv_14', 'bn_14', 'leaky_15', 'shortcut_15', 'conv_16', 'bn_16', 'leaky_17', 'conv_17', 'bn_17', 'leaky_18', 'shortcut_18', 'conv_19', 'bn_19', 'leaky_20', 'conv_20', 'bn_20', 'leaky_21', 'shortcut_21', 'conv_22', 'bn_22', 'leaky_23', 'conv_23', 'bn_23', 'leaky_24', 'shortcut_24', 'conv_25', 'bn_25', 'leaky_26', 'conv_26', 'bn_26', 'leaky_27', 'shortcut_27', 'conv_28', 'bn_28', 'leaky_29', 'conv_29', 'bn_29', 'leaky_30', 'shortcut_30', 'conv_31', 'bn_31', 'leaky_32', 'conv_32', 'bn_32', 'leaky_33', 'shortcut_33', 'conv_34', 'bn_34', 'leaky_35', 'conv_35', 'bn_35', 'leaky_36', 'shortcut_36', 'conv_37', 'bn_37', 'leaky_

[ WARN:0@22284.042] global net_impl.cpp:174 setUpNet DNN module was not built with CUDA backend; switching to CPU


0.6714143753051758
(507, 85)
(2028, 85)
(8112, 85)
[24, 213, 568, 263]
stat['amount_people'] 1


QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6910).
Cannot move to target thread (0x1cc2ef0)

QObject::moveToThread: Current thread (0x1cc2ef0) is not the object's thread (0xfa6

0.3068399429321289
(507, 85)
(2028, 85)
(8112, 85)
[26, 216, 559, 255]
stat['amount_people'] 1
0.2842848300933838
(507, 85)
(2028, 85)
(8112, 85)
[9, 215, 511, 270]
stat['amount_people'] 1
0.3187754154205322
(507, 85)
(2028, 85)
(8112, 85)
[38, 211, 539, 265]
stat['amount_people'] 1
0.29137396812438965
(507, 85)
(2028, 85)
(8112, 85)
[34, 215, 546, 258]
stat['amount_people'] 1
0.32434940338134766
(507, 85)
(2028, 85)
(8112, 85)
[7, 209, 518, 277]
stat['amount_people'] 1
0.280841588973999
(507, 85)
(2028, 85)
(8112, 85)
[1, 212, 526, 273]
stat['amount_people'] 1
0.3005249500274658
(507, 85)
(2028, 85)
(8112, 85)
[21, 211, 569, 261]
stat['amount_people'] 1
0.2988274097442627
(507, 85)
(2028, 85)
(8112, 85)
[28, 209, 558, 267]
stat['amount_people'] 1
0.28892040252685547
(507, 85)
(2028, 85)
(8112, 85)
[-9, 208, 542, 278]
stat['amount_people'] 1
0.28578829765319824
(507, 85)
(2028, 85)
(8112, 85)
[24, 212, 563, 261]
stat['amount_people'] 1
0.28179287910461426
(507, 85)
(2028, 85)
(8112, 85

...

0.2788238525390625
(507, 85)
(2028, 85)
(8112, 85)
[53, 190, 504, 293]
stat['amount_people'] 1
0.2690095901489258
(507, 85)
(2028, 85)
(8112, 85)
[44, 191, 523, 293]
stat['amount_people'] 1
0.25485777854919434
(507, 85)
(2028, 85)
(8112, 85)
[40, 193, 532, 294]
stat['amount_people'] 1
0.24941062927246094
(507, 85)
(2028, 85)
(8112, 85)
[33, 193, 545, 292]
stat['amount_people'] 1
0.26139354705810547
(507, 85)
(2028, 85)
(8112, 85)
[53, 191, 502, 294]
stat['amount_people'] 1
0.2743673324584961
(507, 85)
(2028, 85)
(8112, 85)
[38, 196, 532, 289]
stat['amount_people'] 1
0.3129398822784424
(507, 85)
(2028, 85)
(8112, 85)
[34, 197, 543, 292]
stat['amount_people'] 1
0.26348304748535156
(507, 85)
(2028, 85)
(8112, 85)
[37, 197, 536, 291]
stat['amount_people'] 1
0.2725834846496582
(507, 85)
(2028, 85)
(8112, 85)
[43, 199, 525, 284]
stat['amount_people'] 1
0.3054323196411133
(507, 85)
(2028, 85)
(8112, 85)
[35, 197, 540, 290]
stat['amount_people'] 1
0.3014352321624756
(507, 85)
(2028, 85)
(8112,

0.27231907844543457
(507, 85)
(2028, 85)
(8112, 85)
[130, 194, 453, 293]
stat['amount_people'] 1
0.2818741798400879
(507, 85)
(2028, 85)
(8112, 85)
[115, 192, 483, 296]
stat['amount_people'] 1
0.28682422637939453
(507, 85)
(2028, 85)
(8112, 85)
[110, 193, 494, 292]
stat['amount_people'] 1
0.25892162322998047
(507, 85)
(2028, 85)
(8112, 85)
[123, 192, 463, 293]
stat['amount_people'] 1
0.28307533264160156
(507, 85)
(2028, 85)
(8112, 85)
[120, 196, 469, 290]
stat['amount_people'] 1
0.30124616622924805
(507, 85)
(2028, 85)
(8112, 85)
[124, 196, 466, 288]
stat['amount_people'] 1
0.27431249618530273
(507, 85)
(2028, 85)
(8112, 85)
[127, 194, 456, 291]
stat['amount_people'] 1
0.2775731086730957
(507, 85)
(2028, 85)
(8112, 85)
[127, 198, 463, 289]
stat['amount_people'] 1
0.27076077461242676
(507, 85)
(2028, 85)
(8112, 85)
[133, 193, 447, 298]
stat['amount_people'] 1
0.26859378814697266
(507, 85)
(2028, 85)
(8112, 85)
[129, 193, 454, 297]
stat['amount_people'] 1
0.26852965354919434
(507, 85)
(2

0.2771873474121094
(507, 85)
(2028, 85)
(8112, 85)
[160, 195, 486, 290]
stat['amount_people'] 1
0.2634754180908203
(507, 85)
(2028, 85)
(8112, 85)
[82, 204, 541, 274]
stat['amount_people'] 1
0.3052232265472412
(507, 85)
(2028, 85)
(8112, 85)
[89, 205, 528, 274]
stat['amount_people'] 1
0.32970142364501953
(507, 85)
(2028, 85)
(8112, 85)
[96, 204, 518, 275]
stat['amount_people'] 1
0.3055458068847656
(507, 85)
(2028, 85)
(8112, 85)
[164, 197, 478, 285]
stat['amount_people'] 1
0.3213155269622803
(507, 85)
(2028, 85)
(8112, 85)
[154, 200, 493, 279]
stat['amount_people'] 1
0.28987622261047363
(507, 85)
(2028, 85)
(8112, 85)
[58, 196, 534, 290]
stat['amount_people'] 1
0.3019905090332031
(507, 85)
(2028, 85)
(8112, 85)
[4, 198, 603, 285]
stat['amount_people'] 1
0.3022651672363281
(507, 85)
(2028, 85)
(8112, 85)
[39, 205, 539, 274]
stat['amount_people'] 1
0.34106945991516113
(507, 85)
(2028, 85)
(8112, 85)
[82, 204, 550, 272]
stat['amount_people'] 1
0.3158407211303711
(507, 85)
(2028, 85)
(8112

0.2856135368347168
(507, 85)
(2028, 85)
(8112, 85)
[61, 187, 495, 295]
stat['amount_people'] 1
0.2825038433074951
(507, 85)
(2028, 85)
(8112, 85)
[70, 188, 478, 293]
stat['amount_people'] 1
0.2709517478942871
(507, 85)
(2028, 85)
(8112, 85)
[61, 190, 492, 292]
stat['amount_people'] 1
0.3437316417694092
(507, 85)
(2028, 85)
(8112, 85)
[65, 190, 487, 290]
stat['amount_people'] 1
0.29053306579589844
(507, 85)
(2028, 85)
(8112, 85)
[70, 190, 477, 292]
stat['amount_people'] 1
0.2774801254272461
(507, 85)
(2028, 85)
(8112, 85)
[68, 194, 482, 283]
stat['amount_people'] 1
0.25997328758239746
(507, 85)
(2028, 85)
(8112, 85)
[68, 192, 487, 288]
stat['amount_people'] 1
0.25650501251220703
(507, 85)
(2028, 85)
(8112, 85)
[69, 195, 477, 281]
stat['amount_people'] 1
0.26851916313171387
(507, 85)
(2028, 85)
(8112, 85)
[111, 191, 500, 295]
stat['amount_people'] 1
0.2705497741699219
(507, 85)
(2028, 85)
(8112, 85)
[50, 196, 513, 282]
stat['amount_people'] 1
0.2634093761444092
(507, 85)
(2028, 85)
(8112

0.3068418502807617
(507, 85)
(2028, 85)
(8112, 85)
[6, 198, 524, 278]
stat['amount_people'] 1
0.28791093826293945
(507, 85)
(2028, 85)
(8112, 85)
[8, 205, 597, 270]
stat['amount_people'] 1
0.30302977561950684
(507, 85)
(2028, 85)
(8112, 85)
[5, 196, 523, 281]
stat['amount_people'] 1
0.3139338493347168
(507, 85)
(2028, 85)
(8112, 85)
[6, 206, 600, 268]
stat['amount_people'] 1
0.30663275718688965
(507, 85)
(2028, 85)
(8112, 85)
[8, 202, 595, 276]
stat['amount_people'] 1
0.29064035415649414
(507, 85)
(2028, 85)
(8112, 85)
[7, 206, 602, 268]
stat['amount_people'] 1
0.3048536777496338
(507, 85)
(2028, 85)
(8112, 85)
[5, 196, 528, 281]
stat['amount_people'] 1
0.2753720283508301
(507, 85)
(2028, 85)
(8112, 85)
[1, 199, 529, 276]
stat['amount_people'] 1
0.278947114944458
(507, 85)
(2028, 85)
(8112, 85)
[10, 201, 591, 276]
stat['amount_people'] 1
0.2838418483734131
(507, 85)
(2028, 85)
(8112, 85)
[10, 203, 593, 271]
stat['amount_people'] 1
0.30272555351257324
(507, 85)
(2028, 85)
(8112, 85)
[5,

In [92]:
## distancia
np.sqrt((239-261)**2 + (368-359)**2)

23.769728648009426